In [24]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [25]:
IMG_SIZE = 64
DATA_DIR = "D:\\sign language detection\\dataset\\asl_alphabet_train\\asl_alphabet_train" 

In [26]:
X, y = [], []
labels = sorted(os.listdir(DATA_DIR))
label_map = {label: i for i, label in enumerate(labels)}

In [27]:
for label in labels:
    path = os.path.join(DATA_DIR, label)
    for img_name in os.listdir(path)[:1000]:  # limit per class
        img = cv2.imread(os.path.join(path, img_name))
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        X.append(img)
        y.append(label_map[label])

X = np.array(X) / 255.0
y = to_categorical(y, num_classes=len(labels))
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [28]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping

In [29]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dense(len(labels), activation='softmax')
])


In [30]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [32]:
model.fit(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_val, y_val),
          callbacks=[EarlyStopping(patience=3)])

Epoch 1/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 28s 76ms/step - accuracy: 0.9916 - loss: 0.0247 - val_accuracy: 0.9888 - val_loss: 0.0310
Epoch 2/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 30s 81ms/step - accuracy: 0.9888 - loss: 0.0331 - val_accuracy: 0.9972 - val_loss: 0.0137
Epoch 3/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 30s 82ms/step - accuracy: 0.9937 - loss: 0.0194 - val_accuracy: 0.9907 - val_loss: 0.0303
Epoch 4/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 30s 82ms/step - accuracy: 0.9914 - loss: 0.0236 - val_accuracy: 0.9929 - val_loss: 0.0244
Epoch 5/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 30s 82ms/step - accuracy: 0.9908 - loss: 0.0276 - val_accuracy: 0.9974 - val_loss: 0.0129
Epoch 6/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 30s 84ms/step - accuracy: 0.9959 - loss: 0.0123 - val_accuracy: 0.9924 - val_loss: 0.0337
Epoch 7/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 30s 83ms/step - accuracy: 0.9916 - loss: 0.0242 - val_accuracy: 0.9966 - val_loss: 0.0201
Epoch 8/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 31s 85ms/step - accuracy: 0.9929 - loss: 0.0230 - 

In [33]:
model.save("sign_language_model.h5")

In [34]:
np.save("class_names.npy", np.array(labels))